In [1]:
import pandas as pd
from datetime import datetime
import pm4py
import numpy as np
#from scipy.stats import mannwhitneyu
#import collections
#from statistics import mean
#from statistics import median
from collections import defaultdict
from Levenshtein import distance

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics
#from sklearn.metrics.cluster import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

import copy

In [31]:
log1 = pm4py.read.read_xes("SynLogs\Choice_Long_NEW.gz")
log1["case:concept:name"] = log1["case:concept:name"] + 'C1'
log1['type'] = 'C1'

log2 = pm4py.read.read_xes("SynLogs\Choice_Short_NEW.gz")
log2["case:concept:name"] = log2["case:concept:name"] + 'C2'
log2['type'] = 'C2'


log7 = pm4py.read.read_xes("SynLogs\Choice_Long_X_NEW.gz")
log7["case:concept:name"] = log7["case:concept:name"] + 'C3'
log7['type'] = 'C3'

log8 = pm4py.read.read_xes("SynLogs\Choice_Short_X_NEW.gz")
log8["case:concept:name"] = log8["case:concept:name"] + 'C4'
log8['type'] = 'C4'


###

log3 = pm4py.read.read_xes("SynLogs\Loops_Long.xes")
log3["case:concept:name"] = log3["case:concept:name"] + 'L1'
log3['type'] = 'L1'


log4 = pm4py.read.read_xes("SynLogs\Loops_Short.xes")
log4["case:concept:name"] = log4["case:concept:name"] + 'L2'
log4['type'] = 'L2'


log9 = pm4py.read.read_xes("SynLogs\Loops_Long_X.xes")
log9["case:concept:name"] = log9["case:concept:name"] + 'L3'
log9['type'] = 'L3'


log10 = pm4py.read.read_xes("SynLogs\Loops_Short_X.xes")
log10["case:concept:name"] = log10["case:concept:name"] + 'L4'
log10['type'] = 'L4'

###

log5 = pm4py.read.read_xes("SynLogs\Skips_Long.xes")
log5["case:concept:name"] = log5["case:concept:name"] + 'S1'
log5['type'] = 'S1'


log6 = pm4py.read.read_xes("SynLogs\Skips_Short.xes")
log6["case:concept:name"] = log6["case:concept:name"] + 'S2'
log6['type'] = 'S2'

log11 = pm4py.read.read_xes("SynLogs\Skips_Long_X.xes")
log11["case:concept:name"] = log11["case:concept:name"] + 'S3'
log11['type'] = 'S3'


log12 = pm4py.read.read_xes("SynLogs\Skips_Short_X.xes")
log12["case:concept:name"] = log12["case:concept:name"] + 'S4'
log12['type'] = 'S4'


###

log13 = pm4py.read.read_xes("SynLogs\Choice_Skip_Short_Long_NEW.gz")
log13["case:concept:name"] = log13["case:concept:name"] + 'M1'
log13['type'] = 'M1'


log14 = pm4py.read.read_xes("SynLogs\Choice_Skip_Short_Long_X_NEW.gz")
log14["case:concept:name"] = log14["case:concept:name"] + 'M2'
log14['type'] = 'M2'


log = pd.concat([log1, log2, log3, log4, log5, log6, log11, log12, log10, log9, log8, log7, log13, log14])


parsing log, completed traces :: 100%|██████████| 600/600 [00:00<00:00, 1895.54it/s]


In [32]:
# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'tabName_element' column
log['activity'] = le.fit_transform(log['concept:name'])

In [33]:
#Create trace log
logVar = log.groupby(['case:concept:name'])['activity'].apply(list).reset_index()

In [34]:
logType = log.groupby(['case:concept:name'])['type'].apply(list).reset_index()
logType['type'] = logType['type'].apply(lambda x: set(x))
logType['type'] = logType['type'].apply(lambda x: ''.join(x))

In [35]:
logVar0 = pd.merge(logVar, logType, on='case:concept:name')

In [36]:
logVar0['type'].value_counts()

M1    600
M2    600
C1    300
C2    300
C3    300
C4    300
L1    150
L2    150
L3    150
L4    150
S1    150
S2    150
S3    150
S4    150
Name: type, dtype: int64

In [37]:
#Derive set of unique trace variants + count
logVar0["act_str"] = logVar0["activity"].apply(lambda x: str(x))
#logVar0['Count'] = logVar0.groupby(['act_str'])['activity'].transform('count')
logVar0 = logVar0.drop_duplicates(subset=['act_str'], keep='first')
logVar0 = logVar0.reset_index(drop=True)
logVar0

,case:concept:name,activity,type,act_str
0,case_0C1,"[0, 1, 2, 7, 0, 3, 4, 7]",C1,"[0, 1, 2, 7, 0, 3, 4, 7]"
1,case_0C2,"[0, 1, 5, 6, 7, 0, 4, 5, 6, 7]",C2,"[0, 1, 5, 6, 7, 0, 4, 5, 6, 7]"
2,case_0C3,"[8, 9, 10, 15, 8, 13, 14, 15, 8, 11, 12, 15, 8...",C3,"[8, 9, 10, 15, 8, 13, 14, 15, 8, 11, 12, 15, 8..."
3,case_0C4,"[8, 9, 13, 14, 15, 8, 10, 13, 14, 15]",C4,"[8, 9, 13, 14, 15, 8, 10, 13, 14, 15]"
4,case_0L1,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ...",L1,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ..."
...,...,...,...,...
1152,case_9L1,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ...",L1,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ..."
1153,case_9L2,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ...",L2,"[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, ..."
1154,case_9L3,"[8, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 1...",L3,"[8, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 1..."
1155,case_9L4,"[8, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 1...",L4,"[8, 9, 10, 11, 12, 13, 14, 15, 8, 9, 10, 11, 1..."


In [38]:
logVar0['type'].value_counts()

L1    132
L2    124
L3    123
L4    121
M1    105
M2     88
S4     79
S2     75
C2     63
C4     60
S3     59
S1     51
C1     42
C3     35
Name: type, dtype: int64

In [39]:
#logVar0.loc[logVar0['type'] == 'C1']

In [40]:
n=20

logVar1 = logVar0.loc[logVar0['type'] == 'C1']
logVar1 = logVar1.sample(n)
logVar1['type'] = 'C'

logVar7 = logVar0.loc[logVar0['type'] == 'C3']
logVar7 = logVar7.sample(n)
logVar7['type'] = 'C'



logVar2 = logVar0.loc[logVar0['type'] == 'C2']
logVar2 = logVar2.sample(n)
logVar2['type'] = 'C'

logVar8 = logVar0.loc[logVar0['type'] == 'C4']
logVar8 = logVar8.sample(n)
logVar8['type'] = 'C'


###

logVar3 = logVar0.loc[logVar0['type'] == 'L1']
logVar3 = logVar3.sample(n)
logVar3['type'] = 'L'

logVar9 = logVar0.loc[logVar0['type'] == 'L3']
logVar9 = logVar9.sample(n)
logVar9['type'] = 'L'



logVar4 = logVar0.loc[logVar0['type'] == 'L2']
logVar4 = logVar4.sample(n)
logVar4['type'] = 'L'

logVar10 = logVar0.loc[logVar0['type'] == 'L4']
logVar10 = logVar10.sample(n)
logVar10['type'] = 'L'


###

logVar5 = logVar0.loc[logVar0['type'] == 'S1']
logVar5 = logVar5.sample(n)
logVar5['type'] = 'S'

logVar11 = logVar0.loc[logVar0['type'] == 'S3']
logVar11 = logVar11.sample(n)
logVar11['type'] = 'S'



logVar6 = logVar0.loc[logVar0['type'] == 'S2']
logVar6 = logVar6.sample(n)
logVar6['type'] = 'S'

logVar12 = logVar0.loc[logVar0['type'] == 'S4']
logVar12 = logVar12.sample(n)
logVar12['type'] = 'S'

###

logVar13 = logVar0.loc[logVar0['type'] == 'M1']
logVar13 = logVar13.sample(40)
logVar13['type'] = 'M'

logVar14 = logVar0.loc[logVar0['type'] == 'M2']
logVar14 = logVar14.sample(40)
logVar14['type'] = 'M'

In [41]:
logVar = pd.concat([logVar1, logVar2, logVar3, logVar4, logVar5, logVar6, logVar11, logVar12, logVar10, logVar9, logVar8, logVar7, logVar13, logVar14]).reset_index()
logVar
 

,index,case:concept:name,activity,type,act_str
0,566,case_223C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 5, 6, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 5, 6, 7, 0, 5, 6, ..."
1,538,case_204C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ..."
2,995,case_71C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ..."
3,603,case_251C1,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 5, 6, 7, 0, 3, 4, 7]",C,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 5, 6, 7, 0, 3, 4, 7]"
4,508,case_187C1,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 3, 4, 7, 0, 5, 6, ..."
...,...,...,...,...,...
315,559,case_219M2,"[8, 9, 13, 14, 15, 8, 13, 15, 8, 13, 15, 8, 10...",M,"[8, 9, 13, 14, 15, 8, 13, 15, 8, 13, 15, 8, 10..."
316,454,case_152M2,"[8, 9, 13, 14, 15, 8, 10, 13, 15, 8, 10, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 10, 13, 15, 8, 10, 13, 1..."
317,344,case_135M2,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1..."
318,478,case_163M2,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1..."


In [42]:
logVar['type'].value_counts()

C    80
L    80
S    80
M    80
Name: type, dtype: int64

In [43]:
logVar.to_csv('logVar_PLG_SynL4.csv', index = False)

In [44]:
logVar = pd.read_csv('logVar_PLG_SynL4.csv')

In [45]:
logVar

,index,case:concept:name,activity,type,act_str
0,566,case_223C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 5, 6, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 5, 6, 7, 0, 5, 6, ..."
1,538,case_204C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ..."
2,995,case_71C1,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 5, 6, 7, 0, 3, 4, 7, 0, 5, 6, ..."
3,603,case_251C1,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 5, 6, 7, 0, 3, 4, 7]",C,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 5, 6, 7, 0, 3, 4, 7]"
4,508,case_187C1,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 3, 4, 7, 0, 5, 6, ...",C,"[0, 1, 2, 7, 0, 3, 4, 7, 0, 3, 4, 7, 0, 5, 6, ..."
...,...,...,...,...,...
315,559,case_219M2,"[8, 9, 13, 14, 15, 8, 13, 15, 8, 13, 15, 8, 10...",M,"[8, 9, 13, 14, 15, 8, 13, 15, 8, 13, 15, 8, 10..."
316,454,case_152M2,"[8, 9, 13, 14, 15, 8, 10, 13, 15, 8, 10, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 10, 13, 15, 8, 10, 13, 1..."
317,344,case_135M2,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1..."
318,478,case_163M2,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1...",M,"[8, 9, 13, 14, 15, 8, 11, 12, 13, 15, 8, 13, 1..."


In [46]:
import ast

logVar["activity"] = logVar["activity"].apply(lambda x: ast.literal_eval(x))
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(int(i)) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [47]:
logVar["length"] = logVar["activity"].apply(lambda x: len(x))
logVar['length'].max()

202

In [48]:
logVar['length'].min()

8

## Ground truth 

In [49]:
# Initialize LabelEncoder
le = LabelEncoder()

#Create dictionary with true labels
logVar['class'] = le.fit_transform(logVar['type'])
labelDict1 = logVar['class'].to_dict()
#labelDict1

In [50]:
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(i) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [51]:
def matrix_calc(features, distance):
    n = len(features)
    dist_matrix = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i, n):
            dist_matrix[i,j] = distance(features[i], features[j])
            dist_matrix[j,i] = dist_matrix[i,j]
    
    return dist_matrix

In [52]:
def results(DistMatrix):
    print('NN:   ' + str(NearestNeighbor(DistMatrix, labelDict1)))
    print('P@10: ' + str(PrecisionAtK(DistMatrix, labelDict1, 10)))

    triplet = BatchAllTtripletLoss()
    print('Tri:  ' + str(triplet.forward(DistMatrix,labelDict1)))
    print('Sil:  ' + str(Silhouette(DistMatrix, labelDict1)))

## Evaluation based on Nearest Neighbor

In [53]:
def NearestNeighbor(matrix, labelDict):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    for i in range(len(matrix)):
        #delete 0 in array matrix[i] for distance between (i,i)
        x = np.delete(matrix[i], i) 
        #identify min distance/value in array
        y = min(x)
        
        #identify position of y AND select first position/pair appearing in the array in case there are muliple pairs with identical min distance
        nearestNeighbor = np.where(x == y)[0] #problem if multiple positions??
        nearestNeighbor = int(nearestNeighbor[0])
        
        #true Label for i
        trueLabel = labelDict[i]
        
        #for label comparison add +1 to dict position if position occurs after i (because of the deletion of 0 at the beginning)
        if nearestNeighbor >= i:
            if trueLabel == labelDict[nearestNeighbor + 1]:
                clusterMeasuredCount[trueLabel] += 1         
        else:
            if trueLabel == labelDict[nearestNeighbor]:
                clusterMeasuredCount[trueLabel] += 1
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterMeasuredCount)
    #print(clusterTrueCount)
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 8,  9,  5, 4, 0]])

labelDict2 = {0:1,1:2,2:1,3:3,4:3}
NearestNeighbor(x, labelDict2)
'''

'\nx = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 8,  9,  5, 4, 0]])\n\nlabelDict2 = {0:1,1:2,2:1,3:3,4:3}\nNearestNeighbor(x, labelDict2)\n'

## Evaluation based on Precision@k

In [54]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    #return None  # Return None if the value is not found in the dictionary




def PrecisionAtK(matrix, labelDict, k):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    
    for i in range(len(matrix)):
        
        #true Label for i
        trueLabel = labelDict[i]

        
        #delete 0 in array matrix[i] for distance between (i,i)   
        x = np.delete(matrix[i], i)
        
        #get all k minimum values
        nnValues = []
        for l in range(k):
            y = min(x)
            nnValues.append(y)
            x = np.delete(x, np.where(x == y)[0][0])
        
        
        #get location of minimum values
        z = np.delete(matrix[i], i)
        
        #create dict from array with {position:value}
        my_dict = {}
        for m in range(len(z)):
            my_dict[m] = z[m]
        
        key_list = []
        for n in nnValues:
            key = get_key_by_value(my_dict, n)
            key_list.append(key)
            del my_dict[key]
        
        for o in range(k):
            position = list(key_list)[o]

            if position >= i:
                if trueLabel == labelDict[position + 1]:
                    clusterMeasuredCount[trueLabel] += 1

            else:
                if trueLabel == labelDict[position]:
                    clusterMeasuredCount[trueLabel] += 1

            #transform array to dict
            #get key value from dict
            #compare label
            #remove key+value from dict

            #Need exception in case y == 0 ??
            
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / k / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x3 = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!

labelDict3 = {0:1,1:2,2:1,3:3,4:3}
PrecisionAtK(x3, labelDict3, 2)
'''

'\nx3 = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!\n\nlabelDict3 = {0:1,1:2,2:1,3:3,4:3}\nPrecisionAtK(x3, labelDict3, 2)\n'

## Triplet

see: https://towardsdatascience.com/triplet-loss-advanced-intro-49a07b7d8905

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [56]:
def get_triplet_mask(labels):
    
    # step 1 - get a mask for distinct indices
    ###print('labels', labels)
    # shape: (batch_size, batch_size)
    indices_equal = torch.eye(labels.size()[0], dtype=torch.bool, device=labels.device)
    ###print('equal', indices_equal)
    indices_not_equal = torch.logical_not(indices_equal)
    ###print('not_equal', indices_not_equal)

    # shape: (batch_size, batch_size, 1)
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    ###print('i_not_j - unsqueeze2', i_not_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    ###print('i_not_k - unsqueeze1', i_not_equal_k)
    # shape: (1, batch_size, batch_size)
    j_not_equal_k = indices_not_equal.unsqueeze(0)
    ###print('j_not_k - unsqueeze0', i_not_equal_k)
    # Shape: (batch_size, batch_size, batch_size)
    distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
    ###print('distinct!!!!', distinct_indices)

    # step 2 - get a mask for valid anchor-positive-negative triplets
    # shape: (batch_size, batch_size)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    ###print('labels_equal', labels_equal)
    # shape: (batch_size, batch_size, 1)
    i_equal_j = labels_equal.unsqueeze(2)
    ###print('i_equal_j', i_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_equal_k = labels_equal.unsqueeze(1)
    ###print('i_equal_k', i_equal_k)
    # shape: (batch_size, batch_size, batch_size)
    valid_indices = torch.logical_and(i_equal_j, torch.logical_not(i_equal_k))
    ###print('valid_indices!!!', valid_indices)
    

    # step 3 - combine two masks
    mask = torch.logical_and(distinct_indices, valid_indices)
    ###print('mask!!', mask)
    return mask

    """compute a mask for valid triplets
    Args:
        labels: Batch of integer labels. shape: (batch_size,)
    Returns:
        Mask tensor to indicate which triplets are actually valid. Shape: (batch_size, batch_size, batch_size)
        A triplet is valid if:
        `labels[i] == labels[j] and labels[i] != labels[k]`
        and `i`, `j`, `k` are different.
    """
    
class custom_activation(nn.Module):
    def __init__(self):
        super(custom_activation, self).__init__()
    
    def forward(self, x):
        x[x>0] = 1
        x[x<=0] = 0
        return x


class BatchAllTtripletLoss(nn.Module):
  """Uses all valid triplets to compute Triplet loss

  Args:
    margin: Margin value in the Triplet Loss equation
  """
  def __init__(self, margin=0): #default margin = 0
    super().__init__()
    self.margin = margin
    self.relu = nn.ReLU() #new
    self.custom = custom_activation()
    
  def forward(self, distance_matrix, labels):
    """computes loss value.

    Args:
      embeddings: Batch of embeddings, e.g., output of the encoder. shape: (batch_size, embedding_dim)
      labels: Batch of integer labels associated with embeddings. shape: (batch_size,)

    Returns:
      Scalar loss value.
    """
    # step 1 - convert to tensor format
    distance_matrix = torch.tensor(distance_matrix)
    labels = torch.tensor(list(labels.values()))


    # step 2 - compute loss values for all triplets by applying broadcasting to distance matrix

    # shape: (batch_size, batch_size, 1)
    anchor_positive_dists = distance_matrix.unsqueeze(2)
    # shape: (batch_size, 1, batch_size)
    anchor_negative_dists = distance_matrix.unsqueeze(1)
    # get loss values for all possible n^3 triplets
    # shape: (batch_size, batch_size, batch_size)
    triplet_loss = anchor_negative_dists - anchor_positive_dists + self.margin
    ###print('tl0',triplet_loss)

    # step 3 - filter out invalid or easy triplets by setting their loss values to 0

    # shape: (batch_size, batch_size, batch_size)
    mask = get_triplet_mask(labels)
    ###print('mask', mask)
    triplet_loss *= mask
    ###print(triplet_loss)
    ###print('tl1:', triplet_loss)
    # easy triplets have negative loss values
    
    triplet_loss = self.custom(triplet_loss)
    ###print(triplet_loss)
    #triplet_loss = F.relu(triplet_loss)

    # step 4 - compute scalar loss value by averaging positive losses
    
    triLossNonZero = (triplet_loss != 0).nonzero(as_tuple=True)
    labelTorchUnique = torch.unique(labels, return_counts=True)
    
    nonZero = len(triLossNonZero[0])
    triLossSum = []
    for i in range(nonZero):
        #Identify L_a --> In Class
        t1 = triLossNonZero[0][i]
        labelIn = labels[t1]
        positionIn = int((labelTorchUnique[0] == labelIn).nonzero(as_tuple=False))
        countIn = labelTorchUnique[1][positionIn]

        #Identify L_b --> Out Class
        t3 = triLossNonZero[2][i]
        labelOut = labels[t3]
        positionOut = int((labelTorchUnique[0] == labelOut).nonzero(as_tuple=False))
        countOut = labelTorchUnique[1][positionOut]

        #Calculate loss
        value = (1/countIn)*(1/countIn)*(1/countOut)  
        ###print(countIn)
        ###print(countOut)
        triLossSum.append(value)
    
    #finally divide by |A|^2-|A|
    A = len(labelTorchUnique[0])  
    lossValue = sum(triLossSum) / (A*A-A)
        
    #OLD
    #E_triplet = (1 / (A^2 - A)) *
    #num_positive_losses = (triplet_loss > eps).float().sum()
    #print(num_positive_losses)
    #print(triplet_loss.sum())
    #triplet_loss = triplet_loss.sum() / (num_positive_losses + eps)
    

    return lossValue

## Silhouette

In [57]:
from sklearn import metrics

def Silhouette(distMatrix, labelDict):
    labelDictList = list(labelDict.values())
    return metrics.silhouette_score(distMatrix, labelDictList)

### Levenshtein Distance

In [59]:
#Levenshtein Distance
#from Levenshtein import distance

lev_dis = matrix_calc(logVar["strings"],distance)
results(lev_dis)

NN:   0.95
P@10: 0.8362499999999999
Tri:  tensor(0.5297)
Sil:  -0.07760801031320698


### Normalized Levenshtein Distance

In [60]:
List = logVar["strings"]

n = len(List)
dist_matrix = np.zeros((n,n))    # initialize distance matrix to a square of zeros

for i in range(n):
    for j in range(i, n):
        dist_matrix[i,j] = distance(List[i], List[j]) / max(len(List[i]),len(List[j]))
        dist_matrix[j,i] = dist_matrix[i,j]       # for the symmetric part, no computation

lev_dis_norm = dist_matrix
results(lev_dis_norm)

NN:   0.953125
P@10: 0.86125
Tri:  tensor(0.4315)
Sil:  0.04582165645710614


### Cosine based on 1-gram

In [61]:
#Create 1-gram

def createVector(charList):
    #dtype = [('structure', 'S10'), ('relfrequ', float)]
    arrayList = np.array(charList)
    unique, counts = np.unique(arrayList, return_counts=True)
    #calculate relative frequency
    relFrequList = np.array((unique, counts)).T
    uniqueList = list(unique)
    return relFrequList[relFrequList[:, 0].argsort()]
    #check completeness
    #if 'tree' not in uniqueList:
        #relFrequList = np.append(relFrequList, np.array([['tree', 0]]), axis=0)
        #print(relFrequList)

        
#Change data format from string to list of unique characters
logVar["1-gram"] = logVar["c:n_chr"].apply(lambda x: createVector(tuple(x)))
#logVar


In [62]:
def alignArrays(array1, array2):
    commonSet = set(array1[:,0]).union(array2[:,0])
    
    for i in commonSet:
        if i not in array1[:,0]:
            array1 = np.append(array1, np.array([[i, '0']]), axis=0)
        if i not in array2[:,0]:
            array2 = np.append(array2, np.array([[i, '0']]), axis=0)
    return array1[array1[:, 0].argsort()], array2[array2[:, 0].argsort()]



In [63]:
from scipy.spatial import distance

def cosineDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(int)
    b = Vector2[:,1].astype(int)
    dist_matrix = distance.cosine(a, b)
    return dist_matrix

In [64]:
#Cosine distance based on 1-gram

cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)
results(cos1_dis)

NN:   0.9875
P@10: 0.9321875
Tri:  tensor(0.4453)
Sil:  0.016480904227800533


### Cosine based on 2-gram and 3-gram

In [65]:
#Change data format from string to list of unique characters
#logVar["charList"] = logVar["trace_variant"].apply(lambda x: list(x))
#logVar

def df_list(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList)):
        new = ''.join(extList[i:i+2])
        list_new.append(new)
    del list_new[-1]
    return list_new

#Change data format from string to list of unique characters
logVar["dfList"] = logVar["c:n_chr"].apply(lambda x: df_list(x))
#logVar

logVar["2-gram"] = logVar["dfList"].apply(lambda x: createVector(x))
#logVar

In [66]:
#Cosine distance based on 2-gram

cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)
results(cos2_dis)

NN:   0.996875
P@10: 0.9674999999999999
Tri:  tensor(0.4577)
Sil:  0.07539687748054147


In [67]:
#Create 3-gram

#Change data format from string to list of unique characters
logVar["charList"] = logVar["c:n_chr"].apply(lambda x: list(x))

def df_list2(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList) - 1):
        new = ''.join(extList[i:i+3])
        list_new.append(new)
    del list_new[-1]
    return list_new

In [68]:
logVar["dfList2"] = logVar["charList"].apply(lambda x: df_list2(x))
logVar["3-gram"] = logVar["dfList2"].apply(lambda x: createVector(x))

In [69]:
#Cosine distance based on 3-gram

cos3_dis = matrix_calc(logVar["3-gram"],cosineDist)
results(cos3_dis)

NN:   1.0
P@10: 0.9634375
Tri:  tensor(0.4567)
Sil:  0.11031656900750404


In [104]:
#Aggregation

aggregate1 = cos1_dis + cos2_dis
results(aggregate1)

NN:   1.0
P@10: 0.9693750000000001
Tri:  tensor(0.4555)
Sil:  0.0442347156624748


In [105]:
#Aggregation

aggregate2 = cos1_dis + cos2_dis + cos3_dis
results(aggregate2)

NN:   1.0
P@10: 0.9693750000000001
Tri:  tensor(0.4572)
Sil:  0.06380013397841348


## Euclidean Distance

In [70]:
# Euclidean distance
# see https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy

def euclidDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(float)
    b = Vector2[:,1].astype(float)
    euclidean_dist = np.linalg.norm(a-b)
    return euclidean_dist

In [71]:
#Euclidean distance based on 1-gram

euc1_dis = matrix_calc(logVar["1-gram"],euclidDist)
results(euc1_dis)

NN:   0.940625
P@10: 0.8240624999999999
Tri:  tensor(0.5591)
Sil:  -0.0523146184777661


In [72]:
#Euclidean distance based on 2-gram

euc2_dis = matrix_calc(logVar["2-gram"],euclidDist)
results(euc2_dis)

NN:   0.9812500000000001
P@10: 0.9268749999999999
Tri:  tensor(0.5862)
Sil:  -0.0254352790756025


In [73]:
#Euclidean distance based on 3-gram

euc3_dis = matrix_calc(logVar["3-gram"],euclidDist)
results(euc3_dis)

NN:   0.9875
P@10: 0.9384375
Tri:  tensor(0.6024)
Sil:  -0.013031277433513955


In [74]:
#Aggregation

agg_euc = euc1_dis + euc2_dis
results(agg_euc)

NN:   0.978125
P@10: 0.8984375
Tri:  tensor(0.5750)
Sil:  -0.044013297517589554


### Jaccard based on 1-gram

In [75]:
#Distance based on activity type

def jaccard_similarity(list1, list2):
    s1, s2 = set(list1), set(list2)
    return 1 - len(s1 & s2) / len(s1 | s2)

In [76]:
#Jaccard based on 1-gram
Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
results(Jacc1_dis)

NN:   0.315625
P@10: 0.37875
Tri:  tensor(0.3114)
Sil:  -0.013960385636547851


In [77]:
#Jaccard based on 2-gram
Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)
results(Jacc2_dis)

NN:   0.996875
P@10: 0.9953125
Tri:  tensor(0.4374)
Sil:  0.051203995353748734


In [78]:
#Jaccard based on 3-gram
Jacc3_dis = matrix_calc(logVar["dfList2"],jaccard_similarity)
results(Jacc3_dis)

NN:   1.0
P@10: 0.990625
Tri:  tensor(0.4448)
Sil:  0.11786637224966667


In [79]:
#Aggregation

agg_jacc1 = Jacc1_dis + Jacc2_dis
results(agg_jacc1)

NN:   0.996875
P@10: 0.9665625
Tri:  tensor(0.4286)
Sil:  0.014146771897744482


In [80]:
agg_jacc2 = Jacc1_dis + Jacc2_dis + Jacc3_dis
results(agg_jacc2)

NN:   0.996875
P@10: 0.989375
Tri:  tensor(0.4410)
Sil:  0.042124327982999386


## Graph based measures

In [111]:
#Now consider edge types

In [81]:
# 1. transfer strings (string numbers) to integers --> HERE we need a new encoding!

def intEncoder(character_List):
    return [np.where(np.array(list(dict.fromkeys(character_List)))==e)[0][0]for e in character_List]

logVar["intList"] = logVar["activity"].apply(lambda x: intEncoder(x))

In [82]:
# 2. transfer intList to int_tupleList

#Create tuple lists
def tuple_list(list_of_encodedActivities):
    #list.insert(0, '*')
    #list.append('*')
    list_new = []
    last_element = list_of_encodedActivities[-1]
    for i in range(len(list_of_encodedActivities)):
        new = tuple(list_of_encodedActivities[i:i+2])
        list_new.append(new)
    del list_new[-1]
    if list_of_encodedActivities.count(last_element) == 1: #check wether last activity in trace has some adjancency relation
        list_new.append((last_element,)) ### NOT Correct
    return list_new

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#tuple_list(q)

logVar["int_tupleList"] = logVar["intList"].apply(lambda x: tuple_list(x))
#logVar["int_tupleList"]

In [83]:
# 3. generate Adjacency List

def adj_list(list_of_tuples):
    adj_list_new = {}
    try:
        for node1, node2 in list_of_tuples:
            #print(node1, node2)
            if node1 not in adj_list_new:
                newlist = []
                newlist.append(node2)
                adj_list_new[node1] = newlist
                #print(adj_list3)
        
            else:
                if node2 not in adj_list_new[node1]:
                    #mylist.extend(adj_list3[node1])
                    adj_list_new[node1].append(node2)
                    #print(adj_list3)
                    #adj_list3[node1] = mylist
    
    #in case activity has no adjacent activity - only possible for last activity --> tuple: (lastAct,)
    except ValueError as ve:
        lastValue = list_of_tuples[-1][0] 
        adj_list_new[lastValue] = list()
    return list(adj_list_new.values())

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#l = tuple_list(q)
#adj_list(l)

logVar["int_adjList"] = logVar["int_tupleList"].apply(lambda x: adj_list(x))
#logVar["int_adjList"]

In [84]:
#Now considering length

from collections import deque

def bfs_4(graph, start, end):
    
    graph = {v: k for v, k in enumerate(graph)}
    #print(start, end)
    queue = deque([(start, 0)])
    seen = set()
    while queue:
        #print(queue)
        node, distance = queue.popleft()
        #if not node:
            #print(start, end, queue)
            #print("GRAPH LIST", graph)
        if node in seen:
            continue
        seen.add(node)
        if node == end:
            return distance 
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
        
#x = {0: [0, 1], 1: [2, 1, 0, int], 2:[2], [3: [1, 5, 3, 7], 4: [3], 5: [6, 5], 6: [1, 7], 7: [8, 9, 7], 8: [5, 8, 10], 9: [3]}
#y = [[0, 1, 5], [1, 2], [3, 4], [4, 2], [5, 0], [3, 6], []]
#bfs_4(y, 1, 6)

In [85]:
from collections import defaultdict, deque

def reverse_graph(graph):
    reversed_graph = defaultdict(list)
    for node in graph:
        for neighbor in graph[node]:
            reversed_graph[neighbor].append(node)
    return reversed_graph


def bfs_5(graph, start, end):
    queue = deque([(start, 0)])
    seen = set()
    visited = {}
    while queue:
        node, distance = queue.popleft()
        if node in seen:
            continue
        seen.add(node)
        if node == end: # maybe quicker if adjacent directly checked
            return visited
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
            if adjacent not in visited:
                visited.update({adjacent:distance})

            
def common_ancestors(graph, node1, node2): 
    #remove cross type edge between node1 and node2
    graph[node1].remove(node2) #maybe issue, if graph is changed and not copy of graph?
    graph = {v: k for v, k in enumerate(graph)}
    graphReverse = reverse_graph(graph)
    setNode1 = bfs_5(graphReverse, node1, 0)
    setNode2 = bfs_5(graphReverse, node2, 0)
    if next((a for a in list(setNode1) if a in list(setNode2)), None) == None:
        firstCommonAnces = next((a for a in list(setNode2) if a in list(setNode1)), None)
    else:
        firstCommonAnces = next((a for a in list(setNode1) if a in list(setNode2)), 0)
    
    #uses a hash map to identify the first common ancestor in both lists
    #looks for the first common ancestor in setNode1, which can also be found in setNode2 
    #--> this might not be the closest distance between setNode1 and setNode2
    #--> e.g., for x= [0,1,3,7,5,6] and y= [4,5,7,8,3] 7 might be closest ancestor, although algo detects 3 !
    #distance = setNode1[firstCommonAnces] + setNode2[firstCommonAnces]
    
    
    if firstCommonAnces != None:   # ISSUE: in some cases the firstCommonAnces cannot be detected!
        ancesDistNode1 =  setNode1[firstCommonAnces] + 1 #the edge from node1 to first parent is counted as 0 by algorithm, therefore +1
        ancesDistNode2 =  setNode2[firstCommonAnces] + 1
        numberSkips = abs(ancesDistNode1 - ancesDistNode2)
        numberCross = min(ancesDistNode1, ancesDistNode2)
    else:
        numberSkips, numberCross = (0,1)
    return numberSkips, numberCross
    #if all(x in crossType for x in i):
    

    

#graphList = [[1], [2, 4, 1], [3, 2, 1], [], [5, 4], [5, 4, 6], [7], []]
#c = [[1, 4], [2], [3], [0, 5], [3, 5], []]
#c2 = {v: k for v, k in enumerate(c)}
#common_ancestors(c, 4, 5)
#reverse_graph(c2)

In [86]:
#Create List for decoding traces
from collections import OrderedDict
logVar["indexList"] = logVar["activity"].apply(lambda x: list(OrderedDict.fromkeys(x)))

### Cosine Edge Type + length (no df relations)

In [87]:
class Graph1:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = [['sequ', 1]]
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return np.array(self.structural_array)
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                self.structural_array[0][1] += 0
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append(['1back ',1])
                    #self.structural_array.append(['back ',1])
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append(['2back ',2])
                    #self.structural_array.append(['back ',2])
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append([str(x+1)+'back ',x+1])
                    #self.structural_array.append(['back ',x+1])
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append([str(y-1)+'forward ',y-1])
                #self.structural_array.append(['forward ' ,y-1])
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append(['forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]),numberSkips])
                self.structural_array.append([str(numberCross)+'cross ',numberCross])
                #self.structural_array.append(['cross ' ,numberCross])
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1



In [88]:
from collections import Counter

def transform_list_of_pairs(pairs):
    return [pair[0] for pair in pairs]



def count_entries(input_list):
    # Count the occurrences of each unique entry in the list
    counter = Counter(input_list)
    
    # Create a NumPy array from the counter dictionary
    result = np.array([[key, count] for key, count in counter.items()], dtype=object)
    
    return result

# Example usage
#input_list = ['sequ', '2back', '2back']
#result = count_entries(input_list)
#print(result)

logVar["int_strucLengthList2"] = logVar.apply(lambda x: Graph1(x.int_adjList, x.indexList).dfs(), axis =1)
logVar["relFrequVec1"] = logVar["int_strucLengthList2"].apply(lambda x: transform_list_of_pairs(x))
logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [89]:
#Cosine distance based on edge types
cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
results(cos_graph_dis)

NN:   1.0
P@10: 0.9796875
Tri:  tensor(0.8143)
Sil:  0.42330117342623563


In [90]:
#Cosine distance based on edge types
agg_cos = cos1_dis + cos2_dis + cos_graph_dis
results(agg_cos)

NN:   1.0
P@10: 0.9921875
Tri:  tensor(0.6659)
Sil:  0.13056350209757522


### Jaccard Edge Type and length + df relation

In [91]:
class Graph2:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = []
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                #self.structural_array[0][1] += 0
                #self.structural_array.append('tree ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ')
                self.structural_array.append('tree')
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(1))
                    #self.structural_array.append('back ' + str(1))
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(2))
                    #self.structural_array.append('back ' + str(2))
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(x+1))
                    #self.structural_array.append('back ' + str(x+1))
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(y-1))
                #self.structural_array.append('forward' + str(y-1))
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberSkips))
                self.structural_array.append('cross ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberCross))
                #self.structural_array.append('cross ' + str(numberCross))
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1



In [92]:
logVar["int_strucLengthList3"] = logVar.apply(lambda x: Graph2(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar

In [93]:
#Jacc similarity based on edge types

jacc_graph = matrix_calc(logVar["int_strucLengthList3"],jaccard_similarity)
results(jacc_graph)

NN:   0.99375
P@10: 0.9425000000000001
Tri:  tensor(0.6234)
Sil:  0.20391353704995793


In [94]:
#Jacc sim based on edge types
agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
results(agg_cos)

NN:   1.0
P@10: 0.9921875
Tri:  tensor(0.6659)
Sil:  0.13056350209757522


## Eventually Follows

In [95]:
#Spatial distance between strings


from scipy.spatial import distance


def distanceSpatial(traceString, char1, char2):
    positions_letter1 = [pos for pos, char in enumerate(traceString) if char == char1]
    positions_letter2 = [pos for pos, char in enumerate(traceString) if char == char2]
    
    distList = []
    

    for i in range(len(positions_letter1)):
        for j in range(len(positions_letter2)):
            dist = positions_letter2[j] - positions_letter1[i]
            if dist > 0:
                    #print(dist)
                distList.append(dist)
                    
    
    if not distList: #distList.append(0) #in the case the char1 is after char2 asign dist 0, i.e. char2 cannot be reached from char1
        return 0
    else:
        return 1/min(distList)





def commonDistance(trace1, trace2):
    
    commonSet = set(trace1) & set(trace2)

    commonList = list(commonSet)
    commonList.sort()
    #print(commonList)

    n = len(commonSet)
    dist_matrix1 = np.zeros((n,n))
    dist_matrix2 = np.zeros((n,n))

    for i in range(n):
        for j in range(i, n):
            dist_matrix1[i,j] = distanceSpatial(trace1, commonList[i], commonList[j])
        
    for i in range(n):
        for j in range(i, n):
            dist_matrix2[i,j] = distanceSpatial(trace2, commonList[i], commonList[j])
    
    #print(dist_matrix1, dist_matrix2)
    return distance.cosine(dist_matrix1.ravel(), dist_matrix2.ravel())



#x = 'ABCDEF'
#y = 'ABCDEBCDEBCDEF'
#z = 'ABCDEBCDEF'
#print(dist_matrix)
#distanceSpatial(x, 'A', 'E')
#listVec = logVar["strings"]
#x= listVec[0]
#y= listVec[1]
#commonDistance(x, y)

In [96]:
dist_matrix_evFollows = matrix_calc(logVar["strings"],commonDistance)
dist_matrix_evFollows = np.nan_to_num(dist_matrix_evFollows, nan=0)
agg_evFollows = 0.7*dist_matrix_evFollows + 0.3*cos1_dis 
results(agg_evFollows)

C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


NN:   1.0
P@10: 0.9903125
Tri:  tensor(0.4511)
Sil:  0.07839009303944726


## Maximal Repeat

In [97]:
from suffix_tree import Tree

In [98]:
#tree.maximal_repeats
def maxRepeat(tree):
    mrList=[]
    for C, path in sorted(tree.maximal_repeats()):
        mrList.append(str(path))
    return mrList

#test_tree = Tree({"A": "aaacdcdcbedbccbadbdebdc"})
#maxRepeat(test_tree)

In [99]:
#create vector based on maximal repeats
logVar["mrList"] = logVar["strings"].apply(lambda x: maxRepeat(Tree({"A": x})))
logVar["mrVector"] = logVar["mrList"].apply(lambda x: createVector(tuple(x)))

In [100]:
#Cosine distance based on maxR

cos_mr = matrix_calc(logVar["mrVector"],cosineDist)
results(cos_mr)

NN:   0.9781249999999999
P@10: 0.9321875
Tri:  tensor(0.4390)
Sil:  0.13541501726007765


In [101]:
#Euclidean distance based on maxR

euc_mr = matrix_calc(logVar["mrVector"],euclidDist)
results(euc_mr)

NN:   0.965625
P@10: 0.8534375
Tri:  tensor(0.5641)
Sil:  0.03761836925750636


In [102]:
#Jaccard similarity based on maxR

jacc_mr = matrix_calc(logVar["mrList"],jaccard_similarity)
results(jacc_mr)

NN:   0.9781249999999999
P@10: 0.9303125
Tri:  tensor(0.4386)
Sil:  0.10574590327177298
